<a href="https://colab.research.google.com/github/valkryhx/lora_bnb_int8/blob/main/%E2%80%9CFinetuning_ClueAI_PromptCLUE_base_v1_5_No_LoRA_20230419_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#[仅仅在colab上执行，本地环境不用]colab连接google drive，这样可以使用gdrive的存储空间存取文件
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#[仅仅在colab上执行，本地环境不用] （-p flag 表示如果文件夹不存在则）建立文件夹 用于存放后续的模型bin文件和tokenizer等各类文件
%mkdir -p /content/gdrive/MyDrive/chatyuan_0419

In [3]:
#[仅仅在colab上执行，本地环境不用] 进入到刚创建的文件夹
%cd /content/gdrive/MyDrive/chatyuan_0419 

/content/gdrive/MyDrive/chatyuan_0419


In [4]:
#查看一下
!ls

# 使用自定义数据集微调chatYuan-v1模型

In [5]:
# 安装需要的包 install libraries 
!pip install sentencepiece
#!pip install transformers
!pip install git+https://github.com/huggingface/transformers.git  #安装使用的是github上的transformer版本
!pip install torch
#!pip install rich[jupyter]
#!pip install peft   #本次的demo不用加速技术 #这个是用于peft加速的 会使用到其中的LoRA低秩适应技术

!pip install datasets
!pip install  "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib #bitsandbytes用于8int量化，因为原始的fp32精度模型加载或训练会OOM
!pip install rouge-score tensorboard py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-3yejs3ia
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-3yejs3ia
  Resolved https://github.com/huggingface/transformers.git to commit 474bf508dfe0d46fc38585a1bb793e5ba74fddfd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=6963021 

In [ ]:
#非jupyter环境可以不用装
!pip install rich[jupyter]

In [6]:
# 引入相应的依赖
import os,json,time
import datasets
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# 导入 T5 modules，chatYuan本质是T5模型
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# rich: 让终端显示的更符合观感，可以不使用
# from rich.table import Column, Table
# from rich import box
# from rich.console import Console
print("end....")

end....


In [7]:
# 查看GPU的信息 GPU：Tesla_T4 显存15G
!nvidia-smi

Thu Apr 20 07:13:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 数据准备和转化

In [8]:
#准备用于微调的句子对 这里准备一个问答句子用于演示
#注意 我们这里使用最直观的jsonl结构 我们会反复多次演示数据集的构造过程以加深印象

s={"question": "[问]篮球+背带裤，告诉我他是谁？[答]", "answer": "你这个小黑子，显然是鸡你太美的那位giegie!"}
raw_data = [s for _ in range(10)]

#fpath,fmode,fencoding
with open('cxk.json', 'w', encoding='utf-8') as file:
  for item in raw_data:
   json_line = json.dumps(item,ensure_ascii=False) #注意这里的ensure_ascii=False 不然文件里面内容都是unicode 极其不可读
   file.write(json_line + "\n")
    
print('数据集文件构造完成 注意查看其中格式进行学习')


数据集文件构造完成 注意查看其中格式进行学习


#数据加载
针对本身就存在train和test的数据集，可以这样加载  
dataset = load_dataset("json" ,data_files={'train':"output1.json",'test':"output2.json"} )
  

如果json是嵌套结构，可以指定json文件中具体的key值对应的value作为加载对象,比如，field='data'  
dataset = load_dataset("json" ,field='data',data_files={'train':"train_data.json" ,"test":'test_data.json'}  )  
如果本身数据集较小，只有一个文件，可以先加载再切分成train和test  
from datasets import load_dataset , load_metric
注意 split='train'不要忘记写  
raw_datasets = load_dataset('json', data_files='./data_samples.json',split='train')  
my_datasets = raw_datasets.train_test_split(test_size=0.2)   
print(my_datasets)  

In [9]:
#采用第三种方式
raw_datasets = load_dataset('json', data_files='./cxk.json',split='train')
my_datasets = raw_datasets.train_test_split(test_size=0.2) #80%比例train 20% test
print(my_datasets)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a4721f00386747ea/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 8
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 2
    })
})


In [ ]:
#训练只要train对应的部分
# train_data = my_datasets['train']
# print(train_data)

In [ ]:
# 查看训练后显存占用情况。如果显存被占用，可以kill掉相关的进程
!nvidia-smi
# !fuser -v /dev/nvidia*

Wed Apr 19 15:23:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
# !nvidia-smi -r 
# 使用以下命令清除训练中残存的GPU显存缓存
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
torch.cuda.empty_cache() 
 

In [ ]:
# 定位调占用显存的进程（后面可以kill掉）
!fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia0:        root       7618 F...m python3
/dev/nvidiactl:      root       7618 F...m python3
/dev/nvidia-uvm:     root       7618 F...m python3


In [ ]:
!kill -9 7618

# 模型下载和保存本地

In [11]:
%pwd

'/content/gdrive/MyDrive/chatyuan_0419'

本次基座模型采用 ClueAI/PromptCLUE-base-v1-5  
hf下载 大小990M 还有附属的一些文件都要下载

In [12]:
model_id = 'base_model_chatyuan_v1'
remote_model = 'ClueAI/PromptCLUE-base-v1-5' 
tokenizer = T5Tokenizer.from_pretrained(remote_model)
model = T5ForConditionalGeneration.from_pretrained(remote_model, device_map="auto")

把模型和tokenizer保存到同一个目录  
这是hf上model card的标准格式

In [13]:
model.save_pretrained(model_id)

In [14]:
tokenizer.save_pretrained(model_id)

('base_model_chatyuan_v1/tokenizer_config.json',
 'base_model_chatyuan_v1/special_tokens_map.json',
 'base_model_chatyuan_v1/spiece.model',
 'base_model_chatyuan_v1/added_tokens.json')

好了 以上准备好了模型和数据


In [35]:
#离线加载
# model_id 是本地路径 刚才已经提前下载mode和tokenzier等文件到本路径
tokenizer = tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

In [ ]:
# print(model)
# #LoRA会在target_modules=['q','v']也就是名为q，v的layer建立可训练的旁路矩阵
# model_peft = get_peft_model(model, peft_config)
# model_peft.print_trainable_parameters()

In [16]:
#注意max_source_length 是token级别的 一个token可能对应多个词  所以别看max source length17好像对应不上原句长度 实际是能对应的
#所有的句子在tokenize之后 都会被处理成固定长度，超过这个长度的会截断，不足这个长度的会padding
#注意 remove_columns的用法，map函数会增加3列（由tokenizer函数实现）,而原有的question和answer列被移除 当然这只是为了选择出合适的长度，并不影响后面的流程
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([my_datasets["train"], my_datasets["test"]]).map(lambda x: tokenizer(x["question"], truncation=True), batched=True, remove_columns=["question", "answer"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 100 percentile of max length for better utilization
max_source_length =int(np.percentile(input_lenghts, 100))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([my_datasets["train"], my_datasets["test"]]).map(lambda x: tokenizer(x["answer"], truncation=True), batched=True, remove_columns=["question", "answer"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 100 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 100)) #max(target_lenghts)
print(f"Max target length: {max_target_length}")
print(my_datasets['test'][0] ,'\n',len(my_datasets['test'][0])) #注意len(my_datasets['test'][0]) 是打印一个dict的len(其实就是kv对的数量) 并不是dict中value的长度

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Max source length: 17


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Max target length: 19
{'question': '[问]篮球+背带裤，告诉我他是谁？[答]', 'answer': '你这个小黑子，显然是鸡你太美的那位giegie!'} 
 2


In [17]:
x=tokenized_targets[0]['input_ids']
print(x ,'\ntoken_len=',len(x))
xx= tokenizer.decode(x)
print(xx,"\nchar_len=",len(xx))
#应该是一个token可以对应多个汉字 所以长度不同 这是符合设计的
#</s>为eos_token 是tokenizer引入的表示结尾的特殊字符 类似的还有bos_token,padding_token

[2264, 129, 51, 364, 85, 3, 6482, 15, 1083, 56, 348, 7098, 15233, 969, 6201, 969, 6201, 37, 1] 
token_len= 19
你这个小黑子，显然是鸡你太美的那位giegie!</s> 
char_len= 28


In [19]:
#定义数据集tokenize处理函数
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    #这里对输入做一个修饰 相当于提示这种格式才用这个风格回答 也就是prompt
    #inputs = ["[问]" + item  +"[答]" for item in sample["question"]  ]
    #句子里面已经有问和答的标记了 这里就不加了
    inputs = [item for item in sample["question"]  ]
    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["answer"], max_length=max_target_length, padding=padding, truncation=True)

    # 把padding的tokenizer.pad_token_id设为-100 后面计算loss的时候根据这个设置的值 忽略补齐字符padding token
    
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [20]:
tokenized_dataset = my_datasets.map(preprocess_function, batched=True, remove_columns=['question' ,'answer'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

# 保存datasets 后续备用 
# 训练文件现在是arrow格式
tokenized_dataset["train"].save_to_disk("data/train")
tokenized_dataset["test"].save_to_disk("data/eval")

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Saving the dataset (0/1 shards):   0%|          | 0/8 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorForSeq2Seq

# 计算loss时忽略pad token，即label_pad_token_id=-100的token不参与loss计算
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 #网上都说这个值比较合适 按照8的倍数进行padding
)

# 模型训练

In [37]:
#使用了auto_find_batch_size=True  也可以自己定batch_size
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

log_output_dir="Log_finetune_chatyuan_v1_0416" #训练日志位置
model_output_dir = 'trained_chatyuan_v1_0419'
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
		auto_find_batch_size=True,
    learning_rate=1e-4, # higher learning rate
    num_train_epochs=100,
    logging_dir=f"{log_output_dir}",
    logging_strategy="steps",
    logging_steps=5,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"], #只用到了train的部分 当然也可以全用
)
model.config.use_cache = False  # silence the warnings. Re-enable for inference

In [38]:
# train model
trainer.train() #这个输出的train_loss 貌似有bug 首次train的最终loss很大 需要训练两次才能正确？

Step,Training Loss
5,4.445600
10,2.158200
15,0.736000
20,0.071500
25,0.010400
30,0.002900
35,0.010300
40,0.000700
45,0.000900
50,0.000300


TrainOutput(global_step=100, training_loss=0.3722438528167549, metrics={'train_runtime': 19.3618, 'train_samples_per_second': 41.319, 'train_steps_per_second': 5.165, 'total_flos': 25678400716800.0, 'train_loss': 0.3722438528167549, 'epoch': 100.0})

In [ ]:
#h=trainer.train() 
#h #这个train_loss总感觉是有bug 第二次训练 其train loss才正常 第一次的train loss总是高出数量级而且和tensorboard对不上

In [39]:
#将训练好的模型保存
trainer.save_model('0419_trained_chatyuan_v1')


In [25]:
#一同保存tokenizer
tokenizer.save_pretrained('0419_trained_chatyuan_v1')

('0419_trained_chatyuan_v1/tokenizer_config.json',
 '0419_trained_chatyuan_v1/special_tokens_map.json',
 '0419_trained_chatyuan_v1/spiece.model',
 '0419_trained_chatyuan_v1/added_tokens.json')

In [40]:
#联系对比一下使用LoRA后的保存方式
# #保存lora模型参数
# peft_model_id="./lora_chatyuan_0327_model"
# trainer.model.save_pretrained(peft_model_id)
# #tokenizer.save_pretrained(peft_model_id)
# # if you want to save the base model to call

# #保存基座模型
# base_model = "./out_base_model_0327" #这个是int8的模型 好像不能直接用 要加载原始的f32的model
# trainer.model.base_model.save_pretrained(base_model)
# tokenizer.save_pretrained(base_model)

In [41]:
# !nvidia-smi -r 
# 使用以下命令清除训练中残存的GPU显存缓存 可能没有啥用
# import torch
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
# torch.cuda.empty_cache() 
 

In [42]:
import torch
from transformers import T5Tokenizer,T5ForConditionalGeneration
from torch import cuda

# 原始的fp32精度加载训练好的模型
#model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
device = 'cuda' if cuda.is_available() else 'cpu'
trained_model = '0419_trained_chatyuan_v1'

model_trained = T5ForConditionalGeneration.from_pretrained(trained_model,  load_in_8bit=False, device_map='auto' )#device_map={"":0}
tokenizer_trained = AutoTokenizer.from_pretrained(trained_model)

model_trained.eval()
print("trained model loaded")


trained model loaded


In [ ]:
#model.config.use_cache = True  # silence the warnings. Re-enable for inference

In [ ]:
# from random import randrange
# sample = my_datasets['test'][randrange(len(my_datasets["test"]))]
# sample

# 训练后的模型infer
加载model_trained 和 tokenizer_trained

In [43]:
from datasets import load_dataset
from random import randrange

model_trained.config.use_cache = True

# Load dataset from the hub and get a sample
#test_dataset = load_dataset("json" ,data_files={"test":'test_data.json'} ,field='data' )
sample = my_datasets['test'][randrange(len(my_datasets["test"]))]['question']

#s1=sample # sample =[问]'篮球+背带裤，告诉我他是谁？'[答]

#试试不要提示词
s1='篮球+背带裤，告诉我他是谁？' 
s2 ="苹果是什么颜色?"
s3 ="请介绍一下深度学习有哪些模型?"
s4 ="请写一个判断数字n奇数还是偶数的python程序。" 
s5 ="写一段python函数，一个判断数字n是否大于100的。"
#s_test=sample["Q"]"
#input_ids = tokenizer(s_test, return_tensors="pt", truncation=True).input_ids.cuda()
list_s = [s1,s2,s3,s4,s5]
#print(list_s)
list_inputs_ids = [tokenizer_trained(s, return_tensors="pt", truncation=True).input_ids.cuda() for s in list_s]
#print(list_inputs_ids)
#top_p 越高 生成结果越多样 temperature越低 分布越冻结趋向于集中的确定性的结果 所以temperature=1.0（默认） or smaller结果就越确定
with torch.inference_mode():
  for input_ids_index in range(len(list_inputs_ids)):
    #print(tokenizer.batch_decode(list_inputs_ids[input_ids_index], skip_special_tokens=True)[0])
    outputs = model_trained.generate(input_ids=list_inputs_ids[input_ids_index], max_new_tokens=100, do_sample=True,top_p=0.9,temperature=1.7)
    print(f"\n{'---'*20}")
    print(f"Q:{list_s[input_ids_index]}")
    print(f"A:{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")


------------------------------------------------------------
Q:篮球+背带裤，告诉我他是谁？
A:你这个小黑子，显然是鸡你太美的那位giegie!你的这个小黑子，显然是鸡你太美的那位gie!

------------------------------------------------------------
Q:苹果是什么颜色?
A:这应该是这个蓝色，它有着银色，但是它的图案是银色的，因为黄色是它的颜色。这颜色就决定了它的颜色范围了!

------------------------------------------------------------
Q:请介绍一下深度学习有哪些模型?
A:哪些深度学习的方法让你拥有更好的用户体验啊？ 首先，你需要学习的几个模式(Auto Learning Turn the Periody)可以学习。如在深度学习中，你需要选择AutoLearning Turn the Periody： Source Pulse Module Problem Calse Defect-Dearning-Through the Data Regular Defect Based

------------------------------------------------------------
Q:请写一个判断数字n奇数还是偶数的python程序。
A:将变量描述为n个正则化处理,把其中数字写成数或偶数之后把数字转换成数,生成n数。在数字格式上进行修改,然后用一个python程序来判断n个公奇项还是偶数。

------------------------------------------------------------
Q:写一段python函数，一个判断数字n是否大于100的。
A:A/B=1:1=99，B是关于3倍体x和0的结果;当D=9=1时,B则被判定比B>200了.


# 没有fintune的模型infer

对比原始没有finetune的model输出

加载model_ori 和
tokenizer_ori

In [44]:
#model_id是最开始保存原始模型的路径
model_ori = T5ForConditionalGeneration.from_pretrained(model_id,  load_in_8bit=False, device_map='auto' )#device_map={"":0}
tokenizer_ori = T5Tokenizer.from_pretrained(model_id)

model_ori.eval()
print("original model loaded")

original model loaded


In [45]:
from datasets import load_dataset
from random import randrange

model_ori.config.use_cache = True

# Load dataset from the hub and get a sample
#test_dataset = load_dataset("json" ,data_files={"test":'test_data.json'} ,field='data' )
sample = my_datasets['test'][randrange(len(my_datasets["test"]))]['question']

s1='篮球+背带裤，告诉我他是谁？' #"你觉得抖音公司会对接化发的梗进行限流吗？"
#s1 = "###Question###" + s1 +"###Answer###" #与训练时的prompt保持风格一致
s2 ="苹果是什么颜色?"
s3 ="请介绍一下深度学习有哪些模型?"
s4 ="请写一个判断数字n奇数还是偶数的python程序。" 
s5 ="写一段python函数，一个判断数字n是否大于100的。"
#s_test=sample["Q"]"
#input_ids = tokenizer(s_test, return_tensors="pt", truncation=True).input_ids.cuda()
list_s = [s1,s2,s3,s4,s5]
#print(list_s)
list_inputs_ids = [tokenizer_ori(s, return_tensors="pt", truncation=True).input_ids.cuda() for s in list_s]
#print(list_inputs_ids)
#top_p 越高 生成结果越多样 temperature越低 分布越冻结趋向于集中的确定性的结果 所以temperature=1.0（默认） or smaller结果就越确定
with torch.inference_mode():
  for input_ids_index in range(len(list_inputs_ids)):
    #print(tokenizer.batch_decode(list_inputs_ids[input_ids_index], skip_special_tokens=True)[0])
    outputs = model_ori.generate(input_ids=list_inputs_ids[input_ids_index], max_new_tokens=100, do_sample=True,top_p=0.9,temperature=1.7)
    print(f"\n{'---'*20}")
    print(f"Q:{list_s[input_ids_index]}")
    print(f"A:{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")


------------------------------------------------------------
Q:篮球+背带裤，告诉我他是谁？
A:这位球迷先生: 您好， 背带裤啊。这位先生。 这是球啊 我: 您把篮球交给他。 这是球啊 我： 先生。这是怎么回事？您好，先生。 这事儿我们暂且讲点有趣的事儿。 足球是运动员通过战术来实现的，我们只能凭经验和智慧进行竞技的。比赛是运动员需要付出很大的努力，我们需要的不只是经验和付出，更是要学会赢得机会的能力，我们需要的除了比赛才能

------------------------------------------------------------
Q:苹果是什么颜色?
A:苹果是什么颜色？这就要到官网，官网上才能见到苹果的外观，是的，颜色是这样的! 颜色和图片不一样，要多加说明，颜色上要有蓝色才可以进行颜色调整了。

------------------------------------------------------------
Q:请介绍一下深度学习有哪些模型?
A:我们今天从两个关键词分析入手总结一下深度学的基本理念，看看深度学习是什么吧。我有一个朋友正在思考这个领域内的创业问题。我今天讨论了很久。发现在传统线下领域里面我看不到的大部分知识框架都是基于碎片时间与深度的学习路径。从这个阶段，我们发现，很多东西都会因为时空的长度等而延迟到大脑。我们希望这个方法能够将人的思考变得容易，并能实现从知识上获得知识的速度。深度学习方法

------------------------------------------------------------
Q:请写一个判断数字n奇数还是偶数的python程序。
A:n和 x (b) (c)()[ k=n = 3] where are given in case that you make sense your myself core circuit, and humiliats when your x multipliers twere there in biomarket. (F) [ m'rk] (m's k.) may first instant

------------------------------------------------

从上面的输出可以看到  
在【篮球+背带裤，告诉我他是谁？】这个问题上  
finetune让模型学到了新的知识
不过fintune会让模型出现遗忘知识的现象  
这可能跟模型参数量有关系  
参数越多 越记得牢  
另外这个模型写代码的能力较弱
